# zhsegment

In [1]:
from zhsegment import *
%load_ext autoreload
%autoreload 2

import logging
import numpy as np

from zhsegment_check import fscore
from collections import Counter
from sklearn.linear_model import LinearRegression

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Objective
* #### Implement iterative approach to design a baseline unigram language model over Chinese words. 

* #### Explore additional ways to improve the F-score of baseline model. Experiment with smoothing techniques and bigrams.

## Iterative Model
_Recursion is computationally inefficient. So we intend to replace recursions with iterations and design a segmenter that uses unigram probabilities to segment words(sequence of Chinese characters) from sentences._
* We define the Entry class to store new entries into the heap. Each entry represents one possible word in the segmented text. 
* We used python in-built library heapq to implement heap push and pop operation on a list. Pop operation removes the Entry with the highest value of log probability from the heap.
* In class Segment, as we iterate over each sentence, we try to find the set of Entries that results in the highest log-probability of the entire sentence. Interim solutions of the dynamic programming algorithm as given in the assignment are stored in Chart array. The final solution is printed by traversing back from the last element of the Chart array.

# Improve F-score

We tried the following approaches:
1. Deal with missing words in the corpus.
2. Apply Good-Turing Smoothing.
3. Experiment with Bigram model.

Lets discuss each of these approaches in the following sections.


### 1. Deal with missing words in the corpus

**a. Assumes that unknown words can only be on length one**
The first enhancement to the default unigram model is done through clever handling of the missing words by modifying the Pdist class. The call function is modified such that it returns the probability 1/N if the missing token is of length 1 otherwise none. This results in segmentation of missing token as separate words. We adopt this technique because Chinese words of smaller length are more likely than larger. 

#### Iterative unigram model which assumes missing words can only be of length 1 having probability 1/N gave an accuracy of 85%

In [2]:
class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.values()))
        self.missingfn = missingfn or (lambda k, N: 1./N)
    
    def __call__(self, key): 
        if key in self: return self[key]/self.N
        elif len(key) == 1: return self.missingfn(key, self.N)
        else: return None

**b. Assumes that unknown words can only be on length two and penalize longer strings** The second enhancement to this model was to allow missing words to attain maximum length of two and add a penalizing function similar to hw0.

_Our Penalty Function:_ $ factor/(N * factor^{length(unk)}) $

#### This enhancement further improved our accuracy of 0.91

In [3]:
FACTOR = 50

def penalize_long_words(word, N):
    return float(FACTOR) / (N * FACTOR ** len(word))


class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.values()))
        self.missingfn = missingfn

    def __call__(self, key):
        if key in self: return self[key]/self.N
        elif len(key) <= 2: return self.missingfn(key, self.N)
        else: return None
        

Pw = Pdist(data=datafile("../data/count_1w.txt"), missingfn=penalize_long_words)
segmenter = Segment(Pw) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("../data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3]))


with open('../data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白国 良、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。


score: 0.91


## 2. Apply Good-Turing Smoothing

Good-Turing is given by the following equation:

$ P_{known} = (c+1)(N_{c+1}) / (N_c * N) $

$ P_{unknown} = (N_{1}) / N $

_When we tried this method for smoothing, we faced the issue where $ N_{c+1} $ would become zero because it doesn't necessarily mean that a type with a count of $ c+1 $ would always be seen in the corpus. To fix that issue, we approached it as follows:_


### a. Interpolation

We implemented a linear regression model in order to estimate the values where $ N_{c+1} $ was zero. Doing so allowed us to plugin a more realistic value rather than 1.

Following run shows it with grid search to get the best penalizing factor

In [4]:
class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        tokens = []
        counts = []
        bk = {}
        for key, count in data:
            tokens.append(key)
            bk[key] = bk.get(key, 0) + int(count)
            
        self.N = float(N or sum(bk.values()))
        
        C = Counter(bk)
        Nc = Counter(list(C.values()))
        
        assert(self.N == sum([k * v for k, v in Nc.items()]))
        
        self.default_value = Nc[1] / self.N
        
        self.missingfn = missingfn or (lambda *args: self.default_value)
        
        types = C.keys()
        
        # Given c, predict Nc
        x = np.array(list(Nc.keys())).reshape((-1, 1))
        y = np.array(list(Nc.values()))
        
        # Disabling intercept will avoid any negative values which messes up the log prob
        self.model = LinearRegression(fit_intercept=False).fit(x, y)
        
        for _type in types:
            c = C[_type]
            
            if Nc[c+1] == 0:
                Ncplusone = self.model.predict(np.array([c+1]).reshape(-1, 1))[0]
            else:
                Ncplusone = Nc[c+1]
                
            logger.debug(f"c:{c} Ncplusone:{Ncplusone}")
            
            c_star = ((c+1) * Ncplusone) / (Nc[c])
            self[_type] = c_star / self.N
        
    def __call__(self, key):
        if key in self: 
            return self[key]
        elif len(key) <= 2:
            return self.missingfn(key, self.N)
        else: 
            return None
        
logger.setLevel(logging.INFO)

for i in range(100, 500, 50):
    
    FACTOR = i

    def penalize_long_words(word, N):
        return float(FACTOR) / (N * FACTOR ** len(word))

    Pw = Pdist(data=datafile("../data/count_1w.txt"), missingfn=penalize_long_words)
    segmenter = Segment(Pw) # note that the default solution for this homework ignores the unigram counts
    output_full = []
    with open("../data/input/dev.txt") as f:
        for line in f:
            output = " ".join(segmenter.segment(line.strip()))
            output_full.append(output)
#     print("\n".join(output_full[:3]))


    with open('../data/reference/dev.out', 'r') as refh:
        ref_data = [str(x).strip() for x in refh.read().splitlines()]
        tally = fscore(ref_data, output_full)
        print("FACTOR:{} score: {:.2f}".format(FACTOR, tally), file=sys.stderr)

FACTOR:100 score: 0.90
FACTOR:150 score: 0.90
FACTOR:200 score: 0.91
FACTOR:250 score: 0.91
FACTOR:300 score: 0.91
FACTOR:350 score: 0.91
FACTOR:400 score: 0.91
FACTOR:450 score: 0.91


However, the score obtained with interpolation was lower than the following method:


### b. Replace 0 terms with 1: 

If $ N_{c+1} $ is zero, replace it with 1 to avoid making the whole term zero. So far the best score we obtained was using this method.

In [5]:
class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        tokens = []
        counts = []
        bk = {}
        for key, count in data:
            tokens.append(key)
            bk[key] = bk.get(key, 0) + int(count)
            
        self.N = float(N or sum(bk.values()))
        
        C = Counter(bk)
        Nc = Counter(list(C.values()))
        
        assert(self.N == sum([k * v for k, v in Nc.items()]))
        
        default_value = Nc[1] / self.N
        
        self.missingfn = missingfn or (lambda *args: default_value)
        
        types = C.keys()
        
        for _type in types:
            c = C[_type]
            c_star = ((c+1) * (Nc[c+1] or 1)) / (Nc[c])
            self[_type] = c_star / self.N
        
    def __call__(self, key):
        if key in self: 
            return self[key]
        elif len(key) <= 2:
            return self.missingfn(key, self.N)
        else: 
            return None
        
logger.setLevel(logging.INFO)

FACTOR = 1000

def penalize_long_words(word, N):
    return float(FACTOR) / (N * FACTOR ** len(word))

Pw = Pdist(data=datafile("../data/count_1w.txt"), missingfn=penalize_long_words)
segmenter = Segment(Pw) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("../data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3]))


with open('../data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白 国 良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。


score: 0.92


##### Further experiments with Good-Turing

In this experiment, we implemented Good-Turing Smoothing with a small change.

The default equation for missing type can be given by $ 𝑃_{𝑢𝑛𝑘𝑛𝑜𝑤𝑛} = 𝑁_1/𝑁 $ 

Instead of doing that, we penalized longer words by doing this instead: $ 𝑃_{𝑢𝑛𝑘𝑛𝑜𝑤𝑛} = 𝑁_{1}/(𝑁∗𝐹^{𝑙𝑒𝑛(𝑡𝑦𝑝𝑒)}) $

With this implementation, and supporting an unknown word with a maximum length of 2, we were also able to get a score of 0.92 on dev.

_Experimentations with Good-Turing can be found in temp directory in `good-turing.ipynb` notebook._

## 3. Experimented with Bigram model

We implemented the bigram model in hope of better resutls on the test set, however it's performance was suboptimal. We obtained accuracy of 0.85 on the dev set, which is the same as the baseline.

## Our final model

We chose the iterative unigram model with Good-Turing smoothing (replacing 0 terms with 1 method) to obtain the final score of **0.92** on dev.

In [6]:
with open("../data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3]))


with open('../data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白 国 良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。


score: 0.92


### Conclusion

We can finally conclude that our unigram method along with smoothing produced the best results. Given more time, we would have experimented with global linear models or neural networks or bidirectional RNNs or transition-based neural language models. 